In [50]:
import pandas as pd
import uuid
import dotenv
import os
from pinecone import Pinecone
from openai import OpenAI
from pinecone import ServerlessSpec
from pinecone_text.sparse import BM25Encoder
import time
import json

dotenv.load_dotenv()
pinecone_api = os.getenv('PINECONE_API_KEY')
rds_password = os.getenv('RDS_PASSWORD')
rds_host = os.getenv('RDS_HOST')
pc = Pinecone(api_key=pinecone_api)


In [51]:
with open('data.json', 'r') as file:
    data = json.load(file)
df = pd.read_csv('mnt/efs/data_pre_v3.csv')

In [54]:
df = df.dropna(subset='Krótki opis')

In [56]:
bm25 = BM25Encoder()
bm25.fit(df['Krótki opis'])

sparse = bm25.encode_documents([text for text in df['Krótki opis']])

Python(4847) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
100%|██████████| 13240/13240 [00:28<00:00, 459.38it/s]


In [57]:
dense = [item['vector'] for item in data]

Python(4849) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 11544 tokens (11544 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
meta_dict = [{'id':id} for id in df['uuid'].tolist()]

upserts = []
for _id, sparse_, dense_, meta in zip(df['uuid'].tolist(), sparse, dense, meta_dict):
    upserts.append({
            'id': _id,
            'sparse_values': sparse_,
            'values': dense_,
            'metadata': meta
        })

In [ ]:
index_name = "tender-ai-hybrid-mmlw"
cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
index.upsert(upserts)

time.sleep(5)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
query = "CZĘŚĆ NR 1 gmina Mirsk"
filter_ids = ['69ecacc3-6ffd-4b7e-9925-a2013c4ece64', 'e0926b0c-66d3-41d8-b184-c98b7b722cda']

# create sparse and dense vectors
sparse_query = bm25.encode_queries(query)
dense_query = get_embedding(query)
# search
result = index.query(
    top_k=5,
    filter={
        "id": {"$in":filter_ids},
    },
    vector=dense_query,
    sparse_vector=sparse_query,
    include_metadata=True
)

results = [{
    'id': result['id'],
    'opis': df[df['uuid'] == result['id']]["Krótki opis"],
} for result in result['matches']]
pd.DataFrame(data=results)

,id,opis
0,69ecacc3-6ffd-4b7e-9925-a2013c4ece64,3 CZĘŚĆ NR 1 – gmina Mirsk (długość ogółem:...
1,e0926b0c-66d3-41d8-b184-c98b7b722cda,85 1.\tPrzedmiotem zamówienia jest zakup wr...


In [56]:
pc.delete_index(index_name)

In [15]:
from sqlalchemy import create_engine

# Replace these variables with your own information
rds_host = rds_host
rds_port = '5432'  # usually 5432
rds_dbname = 'postgres'
rds_username = 'postgres'
rds_password = rds_password
table_name = 'tender-ai'

# Create a connection string
connection_string = f"postgresql+psycopg2://{rds_username}:{rds_password}@{rds_host}:{rds_port}/{rds_dbname}"

# Create an engine
engine = create_engine(connection_string)

# Test connection
try:
    # Connect to the PostgreSQL server
    connection = engine.connect()
    print("Connection to PostgreSQL database was successful")
except Exception as e:
    print(f"Error while connecting to PostgreSQL: {e}")
finally:
    # Close the connection
    if 'connection' in locals():
        connection.close()

Connection to PostgreSQL database was successful
